# Аналитика ДТП по предоставленным данным

Заказчиком (Оунером задачи) выступает проект «Карта ДТП» https://dtp-stat.ru/ — некоммерческий проект, посвященный проблеме дорожно-транспортных происшествий в России. Это платформа сбора данных о ДТП, бесплатный и открытый сервис аналитики ДТП.

**Цель проекта** - провести глубокий анализ данных, сформулировать и проверить гипотезы, опираясь на доступные признаки.

**Задачи**
* Провести исследовательский анализ данных о ДТП за 2025 год.
* Сформулировать и проверить не менее трех гипотез, основываясь на имеющихся признаках.
* Провести анализ ДТП по всем датасетам, представленным на сайте.
* Построить дашборд используя любой удобный инструмент, с учетом того, что дашборд может быть опубликован

**Декомпозиция работы:**
* Загрузка датасета и знакомство с данными
* Предобработка данных
* Исследовательский анализ данных
* Проверка гипотез
* Общие выводы
* Подготовка дашборда

## 1. Загрузка датасета и знакомство с данными

Загрузим необходимые библиотеки:

In [1]:
import os
import pandas as pd
import numpy as np
import gc

# Настройка отображения Jupiter
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)  # Отображать все строки
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Отображать полную ширину столбцов

**Для скачивания датасетов с сайта - необходимо использовать скрипт из ноутбука "DataLoader.ipynb"**

Загрузим данные:

In [2]:
all_records = []

download_dir = 'geojson_files'

# перебираем все .geojson в папке
for filename in os.listdir(download_dir):
    if filename.endswith(".geojson"):
        path = os.path.join(download_dir, filename)
        try:
            with open(path, encoding="utf-8") as f:
                data = json.load(f)
            accidents = data["features"]

            records = []
            for feature in accidents:
                props = feature["properties"].copy()
                props["geometry"] = feature["geometry"]
                records.append(props)

            all_records.extend(records)
            print(f"Загружено {len(records)} строк из {filename}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

# объединяем в один датафрейм
data = pd.DataFrame(all_records)

print(f"Итого строк во всех файлах: {len(data)}")

# Удалим переменную all_records:
del all_records

Загружено 10307 строк из novgorodskaia-oblast.geojson
Загружено 27892 строк из tiumenskaia-oblast.geojson
Загружено 5855 строк из sevastopol.geojson
Загружено 41694 строк из cheliabinskaia-oblast.geojson
Загружено 7015 строк из respublika-kareliia.geojson
Загружено 25787 строк из volgogradskaia-oblast.geojson
Загружено 20977 строк из vladimirskaia-oblast.geojson
Загружено 2323 строк из magadanskaia-oblast.geojson
Загружено 10181 строк из kurganskaia-oblast.geojson
Загружено 29859 строк из sverdlovskaia-oblast.geojson
Загружено 16533 строк из respublika-dagestan.geojson
Загружено 5274 строк из respublika-tyva.geojson
Загружено 2589 строк из chechenskaia-respublika.geojson
Загружено 12534 строк из arkhangelskaia-oblast.geojson
Загружено 5254 строк из respublika-kalmykiia.geojson
Загружено 13438 строк из tambovskaia-oblast.geojson
Загружено 28453 строк из omskaia-oblast.geojson
Загружено 6748 строк из kabardino-balkarskaia-respublika.geojson
Загружено 4274 строк из iamalo-nenetskii-avtono

Откроем датасет и посмотрим на данные:

In [3]:
data.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,"[{'year': 2012, 'brand': 'KIA', 'color': 'Серый', 'model': 'Venga', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Выезд на полосу встречного движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 4}, {'role': 'Пассажир', 'gender': 'Женский', 'violations': [], 'health_status': 'Не пострадал', 'years_of_driving_experience': None}]}, {'year': 2013, 'brand': 'Прочие марки мотоциклов', 'color': 'Красный', 'model': 'Прочие марки мотоциклов', 'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Управление ТС лицом, не имеющим права на управление ТС'], 'health_status': 'Скончался на месте ДТП до приезда скорой медицинской помощи', 'years_of_driving_experience': None}]}]",1,[],0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,"[{'year': 2011, 'brand': 'KIA', 'color': 'Синий', 'model': 'Rio', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Женский', 'violations': ['Другие нарушения ПДД водителем'], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического прохождения)', 'years_of_driving_experience': 10}]}]",0,[],1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток]",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,"[{'year': 2009, 'brand': 'ВАЗ', 'color': 'Черный', 'model': 'Priora', 'category': 'В-класс (малый) до 3,9 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Оставление места ДТП', 'Несоответствие скорости конкретным условиям движения'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 13}]}]",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на стационарном лечении'}]",1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,"[{'year': 2009, 'brand': 'CHEVROLET', 'color': 'Синий', 'model': 'Cruze', 'category': 'D-класс (средний) до 4,6 м', 'participants': [{'role': 'Водитель', 'gender': 'Мужской', 'violations': ['Непредоставление преимущества в движении пешеходу', 'Другие нарушения ПДД водителем'], 'health_status': 'Не пострадал', 'years_of_driving_experience': 25}]}]",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'violations': [], 'health_status': 'Раненый, находящийся (находившийся) на амбулаторном лечении, либо в условиях дневно

Посмотрим общую информацию о датасете:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440070 entries, 0 to 1440069
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   id                      1440070 non-null  int64 
 1   tags                    1440070 non-null  object
 2   light                   1440070 non-null  object
 3   point                   1440070 non-null  object
 4   nearby                  1440070 non-null  object
 5   region                  1440070 non-null  object
 6   scheme                  1361453 non-null  object
 7   address                 1364914 non-null  object
 8   weather                 1440070 non-null  object
 9   category                1440070 non-null  object
 10  datetime                1440070 non-null  object
 11  severity                1440070 non-null  object
 12  vehicles                1440070 non-null  object
 13  dead_count              1440070 non-null  int64 
 14  participants      

In [4]:
print(f'Исходный датасет содержит {data.shape[0]} записей и {data.shape[1]} признак(а/ов):')

Исходный датасет содержит 1440070 записей и 21 признак(а/ов):


* `id`: идентификатор
* `light`: время суток
* `point`: координаты (гео)
* `nearby`: комментарий по месту происшествия (где произошло)
* `region`: город/район
* `address`: адрес
* `weather`: погода
* `category`: тип ДТП
* `datetime`: дата и время происшествия
* `severity`: тяжесть ДТП/вред здоровью
* `vehicles`: участники – транспортные средства:
    * `year`: год производства транспортного средства
    * `brand`: марка транспортного средства
    * `color`: цвет транспортного средства
    * `model`: модель транспортного средства
    * `category`: категория транспортного средства
    * `role`: роль участника
    * `gender`: пол участника
    * `violations`: нарушения правил участником
    * `health_status`: состояние здоровья участника
    * `years_of_driving_experience`: стаж вождения участника (только у водителей)
* `dead_count`: кол-во погибших в ДТП
* `participants`: участники без транспортных средств (описание, как у участников внутри транспортных средств)
* `injured_count`: кол-во раненых в ДТП
* `parent_region`: родительский регион
* `road_conditions`: состояние дорожного покрытия
* `participants_count`: кол-во участников ДТП
* `participant_categories`: категории участников

Так как датасет имеет сложную структуру данных, где внутри одной записи (как в столбцах `vehicles` и `participants`) содержится информация о разных объектах (транспортное средство и участники ДТП) со своими отдельными свойствами, разобъем исходный датасет на две части - в первой части будет общая информация о ДТП, а во второй информация о транспортных средствах и участниках ДТП:

In [4]:
general_df = data.drop(columns=['vehicles','participants'])

detailed_df = data[['id', 'vehicles', 'participants']]

## 2. Предобработка данных

Задачи данного раздела:

* Привести типы данных в соответствие
* Проверить пропуски в датасете и обработать их
* Проверить наличие явных и неявных дубликатов и обработать их
* По возможности дообогатить данные на основе имеющихся

### 2.1. Предобработка датасета с общей информацией о ДТП

Посмотрим первые пять записей общего датасета:

In [7]:
general_df.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",[],Шимский район,200,"Медведь-Батецкий, 7 км",[Ясно],Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,[Сухое],3,"[Все участники, Мотоциклисты]","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': None, 'long': None}",[],Шимский район,600,"Батецкий-Медведь, 10 км",[Ясно],Опрокидывание,2015-06-11 16:10:00,Легкий,0,1,Новгородская область,[Сухое],1,[Все участники],"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","[Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток]",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",[Ясно],Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","[Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход]",Шимский район,740,"рп Шимск, ул Новгородская, 6",[Пасмурно],Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,[Сухое],2,"[Все участники, Пешеходы]","{'type': 'Point', 'coordinates': [30.721529, 58.210136]}"
4,156240,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",[],Шимский район,610,"Шимск-Волот, 14 км",[Снегопад],Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"[Недостатки зимнего содержания, Заснеженное]",3,[Все участники],"{'type': 'Point', 'coordinates': [30.765452, 58.067255]}"


#### 2.1.1 Подготовка полей к дальнейшему анализу

В полях `tags`, `nearby`, `weather`, `road_conditions`, `participant_categories` содержатся списки, преобразуем данные поля к строковому типу.

Определим перечень полей для преобразования:

In [9]:
list_features = ['tags', 'nearby', 'weather', 'road_conditions', 'participant_categories']

Приведем поля к строковому типу:

In [10]:
for column in list_features:
    general_df[column] = general_df[column].apply(lambda x: x[0] if len(x) == 1 else ', '.join(x) if isinstance(x, list) else np.nan)

Проверим результат:

In [11]:
general_df.head()

,id,tags,light,point,nearby,region,scheme,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,1504730,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.332904, 'long': 30.415138}",,Шимский район,200,"Медведь-Батецкий, 7 км",Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты","{'type': 'Point', 'coordinates': [30.415138, 58.332904]}"
1,156565,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': None, 'long': None}",,Шимский район,600,"Батецкий-Медведь, 10 км",Ясно,Опрокидывание,2015-06-11 16:10:00,Легкий,0,1,Новгородская область,Сухое,1,Все участники,"{'type': 'Point', 'coordinates': [None, None]}"
2,156274,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.295429, 'long': 30.494614}","Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток",Шимский район,840,"д Старый Медведь, Медведь-Батецкий, 1 км",Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.494614, 58.295429]}"
3,156238,Дорожно-транспортные происшествия,"В темное время суток, освещение включено","{'lat': 58.210136, 'long': 30.721529}","Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход",Шимский район,740,"рп Шимск, ул Новгородская, 6",Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы","{'type': 'Point', 'coordinates': [30.721529, 58.210136]}"
4,156240,Дорожно-транспортные происшествия,Светлое время суток,"{'lat': 58.067255, 'long': 30.765452}",,Шимский район,610,"Шимск-Волот, 14 км",Снегопад,Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"Недостатки зимнего содержания, Заснеженное",3,Все участники,"{'type': 'Point', 'coordinates': [30.765452, 58.067255]}"


Удалим переменную `list_features`:

In [12]:
del list_features

Проверим уникальные значения по полю `tags`:

In [14]:
general_df['tags'].unique()

array(['Дорожно-транспортные происшествия'], dtype=object)

Данное поле не несет смысловой нагрузки, удалим его:

In [15]:
general_df = general_df.drop(columns='tags')

Распарсим поле `point`:

In [16]:
# Преобразование словарей в отдельные столбцы
location_df = general_df['point'].apply(pd.Series)

# Объединяем с оригинальным DataFrame
general_df = pd.concat([general_df.drop(columns=['point']), location_df], axis=1)

# Удалим переменную location_df
del location_df

Удалим дублирующий столбец с координатами:

In [17]:
general_df = general_df.drop(columns='geometry')

Фактический точный адрес ДТП для целей анализа малоприменим, в датасете есть данные о координатах происшествия, поэтому удалим колонку с адресом:

In [20]:
general_df = general_df.drop(columns='address')

#### 2.1.2. Обработка пропусков данных

Проверим наличие пропусков в датасете:

In [23]:
data_size = general_df.shape[0]

print(20*'==')
print('Пропуски, кол-во')
print(general_df.isnull().sum())
print(20*'==')
print('Пропуски, %')
print(general_df.isna().sum() / data_size * 100)
print(20*'==')

Пропуски, кол-во
id                            0
light                         0
nearby                        0
region                        0
scheme                    78617
weather                       0
category                      0
datetime                      0
severity                      0
dead_count                    0
injured_count                 0
parent_region                 0
road_conditions               0
participants_count            0
participant_categories        0
lat                        9792
long                       9792
dtype: int64
Пропуски, %
id                        0.000000
light                     0.000000
nearby                    0.000000
region                    0.000000
scheme                    5.459249
weather                   0.000000
category                  0.000000
datetime                  0.000000
severity                  0.000000
dead_count                0.000000
injured_count             0.000000
parent_region             0.0

Проверим какие данные содержатся в поле `scheme`:

In [24]:
general_df['scheme'].unique()

array(['200', '600', '840', '740', '610', '900', None, '070', '910',
       '500', '210', '950', '940', '440', '140', '830', '430', '820',
       '850', '010', '120', '130', '930', '420', '810', '220', '300',
       '030', '880', '060', '410', '050', '090', '920', '710', '860',
       '400', '800', '700', '020', '870', '230', '770', '760', '960',
       '720', '730', '320', '980', '750', '330', '110', '040', '620',
       '970', '630', '780', '100', '190', '340', '310'], dtype=object)

Здесь в основном представлены трехзначные числа в формате строки. На сайте ГИБДД отсутствует информация о кодировании схем ДТП. Заполним пропуски заглушкой "unknown":

In [25]:
general_df['scheme'] = general_df['scheme'].fillna('unknown')

Пропуски по полям координат `lat` и `long`. Так как в процентном соотношении таких пропусков менее 1% - удалим такие записи:

In [26]:
general_df = general_df.dropna(subset=['lat', 'long'])

#### 2.1.3. Обработка типов данных полей

Категориальный тип данных требует меньше памяти по сравнению со строковым типом. Поэтому приведем категориальные столбцы к соответствующему типу данных. Определим список категориальных полей:

In [28]:
general_cat_features = ['region', 'scheme', 'weather', 'category', 'severity', 'parent_region']

Преобразуем поля к категориальному типу данных pandas:

In [29]:
for column in general_cat_features:
    general_df[column] =  general_df[column].astype('category')

# Удалим переменную из памяти
del general_cat_features

Проверим максимальные и минимальные значения числовых полей:

In [33]:
# Отберем числовые поля
general_num_features = general_df.select_dtypes(include=['number']).columns.tolist()

# Посчитаем минимальное и максимальное значение
for feature in general_num_features:
    print(f'Признак: {feature} MIN:{general_df[feature].min()} MAX:{general_df[feature].max()}')

# Удалим переменную из памяти
del general_num_features

Признак: id MIN:1 MAX:1523161
Признак: dead_count MIN:0 MAX:20
Признак: injured_count MIN:0 MAX:106
Признак: participants_count MIN:1 MAX:168
Признак: lat MIN:0.0003 MAX:90.0
Признак: long MIN:-179.474244 MAX:179.640841


Для оптимизации используемой памяти приведем поля `dead_count`, `injured_count` к типу int8:

In [34]:
general_df['dead_count'] = general_df['dead_count'].astype('int8')

general_df['injured_count'] = general_df['injured_count'].astype('int8')

Приведем поле `datetime` к типу даты:

In [42]:
general_df['datetime'] = pd.to_datetime(general_df['datetime'])

Проверим полученный результат:

In [43]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1430278 entries, 0 to 1440069
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   id                      1430278 non-null  int64         
 1   light                   1430278 non-null  object        
 2   nearby                  1430278 non-null  object        
 3   region                  1430278 non-null  category      
 4   scheme                  1430278 non-null  category      
 5   weather                 1430278 non-null  category      
 6   category                1430278 non-null  category      
 7   datetime                1430278 non-null  datetime64[ns]
 8   severity                1430278 non-null  category      
 9   dead_count              1430278 non-null  int8          
 10  injured_count           1430278 non-null  int8          
 11  parent_region           1430278 non-null  category      
 12  road_conditions    

#### 2.1.4. Обработка дубликатов

Проверим наличие дубликатов:

In [37]:
general_df.duplicated().sum()

np.int64(0)

Явных дубликатов (полных совпадений строк не обнаружено).

Проверим дубликаты идентификаторов:

In [38]:
general_df['id'].duplicated().sum()

np.int64(0)

Дубликатов идентифиакторов не обнаружено, все записи имеют уникаьный идентификатор.

#### 2.1.5. Дообогащение данных

Добавим отдельные поля с днем, годом, месяцем и часом возникновения ДТП:

In [44]:
general_df['year'] = general_df['datetime'].dt.year
general_df['month'] = general_df['datetime'].dt.month
general_df['day'] = general_df['datetime'].dt.day
general_df['hour'] = general_df['datetime'].dt.hour

In [ ]:
 1   light                   1430278 non-null  object  
 2   nearby                  1430278 non-null  object  
 5   weather                 1430278 non-null  category
 6   category                1430278 non-null  category
 8   severity                1430278 non-null  category
 11  parent_region           1430278 non-null  category
 12  road_conditions         1430278 non-null  object  
 13  participants_count      1430278 non-null  int64   
 14  participant_categories  1430278 non-null  object

In [45]:
general_df.head()

,id,light,nearby,region,scheme,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,lat,long,year,month,day,hour
0,1504730,Светлое время суток,,Шимский район,200,Ясно,Столкновение,2025-05-01 12:55:00,С погибшими,1,0,Новгородская область,Сухое,3,"Все участники, Мотоциклисты",58.332904,30.415138,2025,5,1,12
2,156274,"В темное время суток, освещение включено","Жилые дома индивидуальной застройки, Нерегулируемый пешеходный переход, Остановка общественного транспорта, Нерегулируемый перекрёсток",Шимский район,840,Ясно,Наезд на пешехода,2022-10-07 20:30:00,Тяжёлый,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы",58.295429,30.494614,2022,10,7,20
3,156238,"В темное время суток, освещение включено","Административные здания, Многоквартирные жилые дома, Нерегулируемый пешеходный переход",Шимский район,740,Пасмурно,Наезд на пешехода,2022-12-06 17:05:00,Легкий,0,1,Новгородская область,Сухое,2,"Все участники, Пешеходы",58.210136,30.721529,2022,12,6,17
4,156240,Светлое время суток,,Шимский район,610,Снегопад,Съезд с дороги,2021-11-27 15:00:00,Легкий,0,1,Новгородская область,"Недостатки зимнего содержания, Заснеженное",3,Все участники,58.067255,30.765452,2021,11,27,15
5,156242,"В темное время суток, освещение отсутствует",,Шимский район,600,Пасмурно,Опрокидывание,2017-10-22 20:03:00,Легкий,0,1,Новгородская область,Мокрое,3,Все участники,58.193410,30.885960,2017,10,22,20


Проверим уникальные значения по полю `weather`:

In [12]:
data['weather'].unique().tolist()

['Ясно',
 'Пасмурно',
 'Снегопад',
 'Метель',
 'Дождь',
 'Туман',
 'Пасмурно, Дождь',
 'Ясно, Температура выше +30С',
 'Пасмурно, Снегопад',
 'Дождь, Снегопад',
 'Ясно, Температура ниже -30С',
 'Пасмурно, Туман',
 'Ясно, Туман',
 'Дождь, Туман',
 'Ясно, Ураганный ветер',
 'Пасмурно, Метель',
 'Снегопад, Метель',
 'Пасмурно, Температура ниже -30С',
 'Метель, Температура ниже -30С',
 'Ясно, Метель',
 'Пасмурно, Ураганный ветер',
 'Ясно, Дождь',
 'Снегопад, Температура ниже -30С',
 'Метель, Ураганный ветер',
 'Дождь, Температура выше +30С',
 'Туман, Снегопад',
 'Дождь, Ураганный ветер',
 'Дождь, Метель',
 'Туман, Температура ниже -30С',
 'Снегопад, Ураганный ветер',
 'Пасмурно, Температура выше +30С',
 'Туман, Метель',
 'Дождь, Температура ниже -30С']

На основе данного поля можно создать несколько категориальных бинарных признаков, выделяющих факты неблагоприятных погодных явлений, например:
* Снегопад
* Метель
* Дождь
* Туман
* Ураганный ветер

Проверим уникальные значения по полю `road_conditions`:

In [13]:
data['road_conditions'].unique().tolist()

['Сухое',
 'Недостатки зимнего содержания, Заснеженное',
 'Мокрое',
 'Заснеженное',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Мокрое',
 'Недостатки зимнего содержания, Гололедица',
 'Обработанное противогололедными материалами',
 'Сухое, Отсутствие освещения',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Дефекты покрытия',
 'Сухое, Неудовлетворительное состояние обочин',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Отсутствие дорожных знаков в необходимых местах, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое, Неправильное применение, плохая видимость дорожных знаков, Неудовлетворительное состояние обочин, Отсутствие тротуаров (пешеходных дорожек)',
 'Отсутствие, плохая различимость горизонтальной разметки проезжей части, Сухое',
 'Сухое, Неправильное применение, плохая

Поле содержит большое количество комбинаций описания состояния дорожного покрытия. Посчитаем сколько уникальных характеристик содержится - разобъем описание на части между запятыми и посчитаем количество уникальных характеристик:

In [14]:
unique_combinations = data['road_conditions'].str.split(', ').explode().unique()
unique_count = len(unique_combinations)
unique_count

49

In [15]:
print(f'Получилось {unique_count} уникальных характеристик состояния дорожного покрытия')

Получилось 49 уникальных характеристик состояния дорожного покрытия


Посмотрим какие характерисстики получились:

In [16]:
unique_combinations

array(['Сухое', 'Недостатки зимнего содержания', 'Заснеженное', 'Мокрое',
       'Отсутствие',
       'плохая различимость горизонтальной разметки проезжей части',
       'Гололедица', 'Обработанное противогололедными материалами',
       'Отсутствие освещения', 'Дефекты покрытия',
       'Неудовлетворительное состояние обочин',
       'Отсутствие дорожных знаков в необходимых местах',
       'Отсутствие тротуаров (пешеходных дорожек)',
       'Неправильное применение', 'плохая видимость дорожных знаков',
       'Иные недостатки',
       'Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта',
       'Со снежным накатом', 'Низкие сцепные качества покрытия',
       'Неисправное освещение', 'Загрязненное', 'Сужение проезжей части',
       'наличие препятствий',
       'затрудняющих движение транспортных средств',
       'плохая различимость вертикальной разметки', 'Неровное покрытие',
       'Отсутствие дорожных ограждений в необходимых местах',
  

Из данных признаков можно выделить следующие ключевые характеристики для создания отдельных категориальных бинарных прзнаков состояния дорожного покрытия:
* Гололедица, заснеженное, снежный накат
* Отсутствие освещения, неисправное освещение, недостаточное освещение,
* Дефекты покрытия, неровное покрытие
* Неисправность светофора, плохая видимость светофора

Удалим переменные unique_combinations, unique_count:

In [17]:
del unique_combinations
del unique_count

Проверим уникальные значения по полю `participant_categories`:

In [18]:
data['participant_categories'].unique()

array(['Все участники, Мотоциклисты', 'Все участники',
       'Все участники, Пешеходы', 'Все участники, Велосипедисты',
       'Все участники, Велосипедисты, Мотоциклисты',
       'Все участники, Пешеходы, Мотоциклисты', '',
       'Все участники, Общ. транспорт',
       'Все участники, Пешеходы, Общ. транспорт', 'Мотоциклисты',
       'Все участники, Пешеходы, Велосипедисты',
       'Все участники, Мотоциклисты, Общ. транспорт',
       'Все участники, Велосипедисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Мотоциклисты', 'Пешеходы',
       'Все участники, Пешеходы, Мотоциклисты, Общ. транспорт',
       'Все участники, Пешеходы, Велосипедисты, Общ. транспорт'],
      dtype=object)

Из данного поля можно выделить следующие категориальные признаки:
* Мотоциклисты
* Пешеходы
* Велосипедисты
* Общ. транспорт

#### 2.1.2 Парсинг полей со словарями

In [ ]:
Поля `vehicles` и `participants` имееют схожую структуру

Распарсим столбец `vehicles`:

### 2.2. Предобработка датасета с детальной информацией об участниках ДТП

Проверим наличие пропусков в датасете:

In [27]:
data_size = data.shape[0]

print(20*'==')
print('Пропуски, кол-во')
print(data.isnull().sum())
print(20*'==')
print('Пропуски, %')
print(data.isna().sum() / data_size * 100)
print(20*'==')

Пропуски, кол-во
id                                                  0
light                                               0
nearby                                              0
region                                              0
scheme                                          78617
address                                         75156
weather                                             0
category_vehicle                                    0
datetime                                            0
severity                                            0
dead_count                                          0
injured_count                                       0
parent_region                                       0
road_conditions                                     0
participants_count                                  0
participant_categories                              0
lat                                              9792
long                                             9792
year       

#### 2.2.3. Пропуски по полям координат `lat` и `long`

Проверим где чаще всего встречаются данные пропуски. Создадим временный столбец с признаком пропуска координат:

In [32]:
data['coord_is_nan'] = data.apply(lambda x: 1 if pd.isna(x['lat']) or pd.isna(x['long']) else 0, axis=1)

In [33]:
without_coords_parent_region = data.query('coord_is_nan==1').pivot_table(index=['parent_region'],
                                          values='coord_is_nan',
                                          aggfunc='sum'
                                         ).sort_values(by='coord_is_nan', ascending=False)

print('Топ 10 регионов по пропущенным данным с координатами ДТП:')
without_coords_parent_region.head(10)

Топ 10 регионов по пропущенным данным с координатами ДТП:


,coord_is_nan
parent_region,
Санкт-Петербург,1195
Тюменская область,1135
Республика Дагестан,810
Тамбовская область,470
Новгородская область,411
Пермский край,382
Севастополь,357
Республика Татарстан (Татарстан),335
Республика Коми,330


In [34]:
without_coords_region = data.query('coord_is_nan==1').pivot_table(
                                          index=['parent_region', 'region'],
                                          values='coord_is_nan',
                                          aggfunc='sum'
                                         ).reset_index().sort_values(by='coord_is_nan', ascending=False)

print('Топ 10 регионов по пропущенным данным с координатами ДТП:')
without_coords_region.head(10)

Топ 10 регионов по пропущенным данным с координатами ДТП:


,parent_region,region,coord_is_nan
597,Санкт-Петербург,Московский район,546
625,Севастополь,Севастополь,357
694,Тюменская область,Тобольск,252
695,Тюменская область,Тобольский район,198
589,Санкт-Петербург,Василеостровский район,176
601,Санкт-Петербург,Приморский район,168
180,Костромская область,Кострома,153
367,Приморский край,Партизанский район,140
534,Республика Татарстан (Татарстан),Казань,128
604,Санкт-Петербург,Центральный район,122


Так как в процентном соотношении таких пропусков менее 1% - удалим такие записи:

In [35]:
data = data.query('coord_is_nan==0').copy()

Удалим временный столбец:

In [36]:
data = data.drop(columns='coord_is_nan')

Удалим переменные `without_coords_parent_region`, `without_coords_region`:

In [37]:
del without_coords_parent_region
del without_coords_region

#### 2.2.4. Пропуски по полю `year`

`year` - год выпуска транспортного средства, переименуем данный столбец, чтобы была понятна суть содержащихся данных:

In [38]:
data.rename(columns={'year': 'year_vehicle_manufactured'}, inplace=True)

Удалять или заполнять отсутствующий год выпуска ТС не будем.

#### 2.2.5. Пропуски по полю `brand`

Пропуски марки ТС заполним заглушкой:

In [39]:
data['brand'] = data['brand'].fillna('unknown')

#### 2.2.6. Пропуски по полю `color`

Пропуски цвета ТС заполним заглушкой:

In [40]:
data['color'] = data['color'].fillna('unknown')

#### 2.2.7. Пропуски по полю `model`

Пропуски в признаке модели ТС заполним заглушкой:

In [41]:
data['model'] = data['model'].fillna('unknown')

#### 2.2.8. Пропуски по полю `category`

Пропуски в признаке категории ТС заполним заглушкой:

In [42]:
data['category'] = data['category'].fillna('unknown')

In [ ]:
#### 2.2.9. Пропуски по полю `category`

In [ ]:
prtpnt_no_vhcl_role                            0.468866
prtpnt_no_vhcl_gender                          3.021728
prtpnt_no_vhcl_violations                      0.468866
prtpnt_no_vhcl_health_status                   0.668231
prtpnt_no_vhcl_years_of_driving_experience    25.047671
prtpnt_in_vhcl_role                           70.785656
prtpnt_in_vhcl_gender                         71.267300
prtpnt_in_vhcl_violations                     70.785656
prtpnt_in_vhcl_health_status                  70.787253

In [ ]:
data_size = data.shape[0]

print(20*'==')
print('Пропуски, кол-во')
print(data.isnull().sum())
print(20*'==')
print('Пропуски, %')
print(data.isna().sum() / data_size * 100)
print(20*'==')

### 2.3. Обработка дубликатов

Проверим количество дубликатов в датасете:

In [ ]:
print(data.dtypes)

In [ ]:
data.duplicated().sum()

Явных дубликатов не обнаружено.

Проверим наличие неявных дубликатов по категорийным полям.

Проверим уникальные значения по полю `light`:

In [ ]:
data['light'].unique().tolist()

Неявных дубликатов по полю `light` не обнаружено.

### 2.3. Дообогащение данных

#### 2.3.1. Добавление координат ДТП, соответствующим формату данных DataLens

Для построения геоточек на картах в DataLens необходимы данные в формате '[55.75222,37.61556]', где первое число - широта, вторая долгота.

Получим сначала отдельно координаты широты и долготы:

In [ ]:
data['longitude'] = data['geometry'].geometry.x
data['latitude'] = data['geometry'].geometry.y

In [ ]:
data.to_csv('krsk.csv')